# Import Modules

In [ ]:
%matplotlib inline
import torch

# Check Available Devices


In [ ]:
if torch.cuda.is_available:
  avDev = torch.device("cuda")
else:
  avDev = torch.device("cpu")
print(avDev)

# Set Seed

In [ ]:
seed = 1
torch.manual_seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic=True

# Utility for Displaying Images, Error Curve

# Load Data

# Data Iterable

# Model Definition

# Hyperparameter

# Training Cycle

# Hyperopt

# Visualization of Results

# Metric Measures